In [4]:
# 8 threads
# 14 tests < 1000 fidx takes around 2h
# !TEST_NAME='jsc' FIDX_LIMIT=1000 python rq1.py # 42 mins
# !TEST_NAME='boa' FIDX_LIMIT=1000 python rq1.py # 58 mins
# !TEST_NAME='commanderkeen' FIDX_LIMIT=1000 python rq1.py # 144 mins
# !TEST_NAME='funky-kart' FIDX_LIMIT=1000 python rq1.py # 155 mins
!FIDX_LIMIT=1000 ONLY_FAIL=True python rq1.py

RQ1
RUNNING: wamr#2861 with 1 functions
FINISHED: wamr#2861 in 0.0005226731300354004 minutes
RUNNING: wasmedge#3018 with 12 functions
FINISHED: wasmedge#3018 in 0.0006904959678649903 minutes
RUNNING: wamr#2789 with 58 functions
FINISHED: wamr#2789 in 2.0017606178919474 minutes
RUNNING: wamr#2450 with 66 functions
FINISHED: wamr#2450 in 0.0005558609962463378 minutes
RUNNING: wamr#2862 with 73 functions
FINISHED: wamr#2862 in 0.0005324681599934896 minutes
RUNNING: wasmedge#3019 with 76 functions
FINISHED: wasmedge#3019 in 0.0005364894866943359 minutes
RUNNING: wasmedge#3076 with 131 functions
FINISHED: wasmedge#3076 in 0.0005415439605712891 minutes
RUNNING: mandelbrot with 167 functions
FINISHED: mandelbrot in 0.000533743699391683 minutes
RUNNING: guiicons with 505 functions
FINISHED: guiicons in 0.00053253968556722 minutes
RUNNING: rguilayout with 510 functions
FINISHED: rguilayout in 0.0005095998446146647 minutes
RUNNING: rguistyler with 575 functions
FINISHED: rguistyler in 0.00052140

In [5]:
short_case = [
    "boa",
    "funky-kart",
    "guiicons",
    "jsc",
    "mandelbrot",
    "rfxgen",
    "rguilayout",
    "rguistyler",
    "riconpacker",
    "sqlgui",
    "wamr#2450",
    "wamr#2789",
    "wamr#2861",
    "wamr#2862",
    "wasmedge#3018",
    "wasmedge#3019",
    "wasmedge#3057",
]
long_case = ["commanderkeen", "rtexviewer", "hydro", "wasmedge#3076"]

!TEST_NAMES={','.join(short_case)} python rq2.py wasm-slice # 5 mins
!TEST_NAMES={','.join(long_case)} python rq2.py wasm-slice # 1 hour

len(testset): 1
CompletedProcess(args='timeout 10s wizard-0d6926f -no-names -mode=int /home/don/wasm-r3/benchmarks/sqlgui/sqlgui.sliced.wasm', returncode=10, stdout='<wasm func #0> +131\n  <wasm func #1> +65779\n    !trap[UNREACHABLE]\n', stderr='')
CompletedProcess(args='timeout 5s wasmtime --invoke main /home/don/wasm-r3/benchmarks/sqlgui/sqlgui.sliced.wasm', returncode=0, stdout='', stderr='')
Interesting!


In [ ]:
!MAX_WORKER=1 python rq2.py wasm-reduce

In [ ]:
!MAX_WORKER=1 python rq2.py wasm-shrink

len(testset): 21
Running in single thread mode
Interesting!
Interesting!
Interesting!
Interesting!
Interesting!
Interesting!
Interesting!
Interesting!
Interesting!
Interesting!
Interesting!
Interesting!
Interesting!
Interesting!
Interesting!
Interesting!
Interesting!
Interesting!
CompletedProcess(args='timeout 10s wizard-0d6926f -no-names -mode=spc /home/don/wasm-r3/benchmarks/commanderkeen/commanderkeen.shrunken.wasm', returncode=10, stdout='!trap[STACK_OVERFLOW]\n', stderr='')
CompletedProcess(args='timeout 5s wasmtime --invoke main /home/don/wasm-r3/benchmarks/commanderkeen/commanderkeen.shrunken.wasm', returncode=0, stdout='', stderr='')
Interesting!
Interesting!


In [18]:
!python rq4.py

In [53]:
import json, re
from heuristics_finder import get_dynamic_fidx
from sanity_check import extract_code_bytes
from evaluation import sh


with open("metrics.json", "r") as f:
    metrics = json.load(f)

tests = list(metrics.keys())
tests.remove('rq4')
for test in tests:
    input_path = f'~/wasm-r3/benchmarks/{test}/{test}.wasm'
    code_size = extract_code_bytes(sh(f'wasm-tools objdump {input_path}'))
    metrics[test]['metadata']['code_size'] = code_size
#     didx = get_dynamic_fidx(f'~/wasm-r3/benchmarks/{test}/{test}.wasm')
#     try:
#         for i in range(metrics[test]['metadata']['function_count']):
#             if metrics[test]['rq1'].get(str(i)) is None:
#                 continue
#             if i in didx:
#                 metrics[test]['rq1'][str(i)]['is-executed'] = False
#             else:
#                 metrics[test]['rq1'][str(i)]['is-executed'] = True
#     except:
#         print(f'test: {test}, i: {i}')

def extract_test_fidx(path):
    # Extract the test name (e.g., 'boa') and the fidx (e.g., 366)
    m = re.search(r'/benchmarks/([\w#-]+)/out/(\d+)/', path)
    if m:
        test_name = m.group(1)
        fidx = int(m.group(2))
        return [test_name, fidx]
    raise Exception(f"Failed to extract test name and fidx from {path}")
unit_path = '/home/don/wasm-r3/benchmarks/boa/out/366/benchmarks/bin_1/replay.wasm'
assert extract_test_fidx(unit_path) == ['boa', 366]

paths = !find ~/wasm-r3/benchmarks/ | rg replay.wasm
for path in paths:
    test_name, fidx = extract_test_fidx(path)
    code_size = extract_code_bytes(sh(f'wasm-tools objdump {path}'))
    try:
        metrics[test_name]['rq1'][str(fidx)]['sliced_code_size'] = code_size
    except:
        pass 

with open("metrics.json", "w") as f:
    json.dump(metrics, f, indent=4)